# <font face='微软雅黑'>Chapter 6 Organing Cython Code</font>
Cython全面支持import语句，其和Python中的import语句意义相同，使我们可以在运行时访问定义在外部纯Python模块中的Python对象，或其他扩展模块中的可被Python访问的对象。如果只有import语句，Cython模块间将不能相互访问对方的cdef、cpdef函数、ctypedef、structs，也将不允许对其他extension types进行C级别访问。

为了解决这个问题，Cython提供了三种文件类型来管理项目中Cython专属以及C-level的部分；以及<font face='consolas'>cimport</font>语句对C-level constructs进行编译时访问，其在definition（.pxd）文件中寻找constructs的声明。三种文件分别为：
- implementation files: 后缀".pyx"，Cython源文件
- definition files: 后缀".pxd",注意不是编译后产生的".pyd"
- include files: 后缀".pxi"

本章将涵盖<font face='consolas'>cimport</font>语句的详细介绍，以及三种文件件的关系，以及如何用他们构建更大的Cython项目。

### I. Cython Implementation (.pyx) and Declaration (.pxd) Files
声明declaration与定义definition的区别
- 声明declaration
    - 对函数和方法：包含函数和方法签名的所有要素，声明类型(cdef or cpdef)、函数或方法的名字、参数列表（包括括号）；<font color='#ff2222'>但不包括结尾的冒号</font>
    - 对cdef class：包括<font face='consolas'>cdef class</font>行(包括冒号)以及extension type的名字，所有属性声明以及方法声明
- 定义definition：construct实现所需的所有要素
    - 对函数和方法：重复声明部分作为定义的一部分
    - 对cdef class：不重复声明属性部分

"simulator.pyx"和"simulator.pxd"具有相同的base name，所有被Cython当做一个namespace来对待。cyhton compiler编译"simulator.pyx"时，会自动探测"simulator.pxd"并使用其中的声明。

What belongs inside a definition file? Essentially, anything that is meant to be publicly accessible to other Cython modules at C level.
- definition file中可以包含：
    - C类型的声明——<font face='consolas'>ctypedef, struct, union, 或enum</font>（Chapter 7）
    - 外部C/C++库的声明——<font face='consolas'>cdef extern</font> blocks（Chapter 7 and 8）
    - <font face='consolas'>cdef</font>和<font face='consolas'>cpdef</font>模块级函数的声明
    - <font face='consolas'>cdef class</font> extension types
    - extension types的<font face='consolas'>cdef</font>属性
    - <font face='consolas'>cdef</font>和<font face='consolas'>cpdef</font>方法
    - C-level <font face='consolas'>inline</font>函数和方法的实现（implementation）
- definition file中**<font color='#ff2222'>不能</font>**包含：
    - Python或<font face='consolas'>non-inline</font> C函数或方法的实现
    - Python类的定义（i.e., regular classes）
    - 在IF或DEF宏以外可以被执行的Python代码

*".pxd"*文件的作用是是的外部implementation file可以通过<font face='consolas'>cimport</font>语句访问*<font face='consolas'>simulator.pyx</font>*中的所有C-level constructs

In [1]:
%%file simulator.pxd
ctypedef double real_t
cdef class State:
    cdef:
        unsigned int n_particles
        real_t *x
        real_t *vt
    cpdef real_t momentum(self)
cpdef run(State st)
cpdef int step(State st, real_t timestep)

Writing simulator.pxd


In [2]:
%%file simulator.pyx
cdef class State:
    def __cint__(...):
        # ...
    def __dealloc__(...):
        # ...
    cpdef real_t momentum(self):
        # ...
def setup(input_fname):
    # ...
cpdef run(State st):
    # ...calls step function repeatedly...
cpdef int step(State st, real_t timestep):
    # ...advance st one time step...
def output(State st):
    # ...

Writing simulator.pyx


### II. The cimport Statement
cimport的语法和import非常相似
- <font face='consolas'>cimport simulator</font>
- <font face='consolas'>cimport simulator as sim</font>
- <font face='consolas'>from simulator cimport State as sim_state, step as sim_step</font>

<font face='consolas'>cimport</font> Python-level对象（如<font face='consolas'>setup</font>函数）以及<font face='consolas'>import</font>C-only声明（如<font face='consolas'>real_t</font>）都会爆出compile-time error。我们被允许<font face='consolas'>import</font>或<font face='consolas'>cimport</font> State extension type或者<font face='consolas'>step cpdef</font>函数,尽管通常推荐使用<font face='consolas'>cimport</font>。当我们<font face='consolas'>import</font> extension types或cpdef函数是，我们将只拥有Python-only access，不能访问private属性或<font face='consolas'>cdef</font>方法，且<font face='consolas'>cpdef</font>方法或函数也将使用更慢的Python wrapper。

definition file中可以包含<font face='consolas'>cdef extern</font> blocks。将这样的声明打包放在它们自己的*".pyd"*文件中很实用，可以方便他们在其他地方的使用。

In [3]:
%%file improved_simulator.pyx
# cimport从simulator.pxd导入。
# 访问发生在C level，编译时。
from simulator cimport State, step, real_t
# import从simulator extension module导入。
# 访问发生在Python level，运行时
from simulator import setup as sim_setup

cdef class NewState(State):
    cdef:
        # ...extra attributes...
        def __cint__(self, ...):
            # ...
        def __dealloc__():
            # ...
def setup(fname):
    # ...call sim_setup and tweak things slightly...
cpdef run(State st):
    # ...improved run that uses simulator.step...

Writing improved_simulator.pyx


In [4]:
%%file _mersenne_twister.pxd
# definition file that contains cdef extern block
cdef extern from "mt19937ar.h":
    # initializes mt[N] with a seed
    void init_genrand(unsigned long s)
    # generates a random number on [0,0xffffffff]-interval
    unsigned long genrand_int32()
    # generates a random number on [0,0x7fffffff]-interval
    long genrand_int31()
    # generates a random number on [0,1]-real-interval
    double genrand_real1()
    # generates a random number on [0,1)-real-interval
    double genrand_real2()
    # generates a random number on (0,1)-real-interval
    double genrand_real3()
    # generates a random number on [0,1) with 53-bit resolution
    double genrand_res53()

Writing _mersenne_twister.pxd


In [5]:
%%file mersenne_test.pyx
from _mersenne_twister cimport init_genrand, genrand_real3
cimport _mersenne_twister as mt
mt.init_genrand(42)
for i in range(len(x)):
    x[i] = mt.genrand_real1()

Writing mersenne_test.pyx


### III. Predefined Definition Files
为了方便，Cython为经常使用的C、C++和Python头文件与定义了definition files。他们被归集到definition file packages中，存放在*Cython*源文件目录下的*Includes*目录下。以下是常用的packages：
- <font face='consolas'>libc</font>：package with *.pxd* files for C standard library
    - <font face='consolas'>stdlib, stdio, math, string, stdint</font>及其他头文件
- <font face='consolas'>libcpp</font>：package with *.pxd* files for C++ standard library
    - <font face='consolas'>string, vector, list, map, pair, set</font>等
- <font face='consolas'>cpython</font>：package with *.pxd* files for the C header files found in the CPython source distribution。提供从Cython访问Python/C API的简单途径。
- <font face='consolas'>numpy</font>：提供访问Numpy/C API的途径(<font color='#ff2222'>Chapter 10</font>)。

#### cimport的方式有：
##### [1]从一个package中cimport一个模块
<br></br>
<font face='consolas'>from libc cimport math</font>

##### [2]从模块（dotted module name）中cimport一个对象
<br></br>
<font face='consolas'>from libc.math cimport sin</font>

##### [3]Multiple named cimports
<br></br>
<font face='consolas'>from libc.stdlib cimport rand, srand, qsort, malloc, free</font>

##### [4]cimport时定义别名（alias）
<br></br>
<font face='consolas'>from libc.string cimport memcpy as c_memcpy</font>

##### [5]cimport C++ STL 模板类
<br></br>
<font face='consolas'>from libcpp.vector cimport vector</font>
<br></br>
<font face='consolas'>cdef vector[int] *vi = new vector[int]\(10)</font>

当我们<font face='consolas'>import</font>和<font face='consolas'>cimport</font>具有相同名字的不同函数时，Cython会报出编译时错误。修复这个错误的方法是使用别名。

In [ ]:
# 下列代码会爆出编译时错误
from libc.math cimport sin
from math import sin

# 使用别名可以避免这个错误
from libc.math cimport sin as csin
from math import sin as pysin

可以同时<font face='consolas'>import</font>和<font face='consolas'>cimport</font>具有相同名字的不同namespace-like objects(模块或Cython包)。但不推荐这样做，建议使用别名。

In [ ]:
# 建议使用别名，虽然不使用别名也没有错
from libc cimport math as cmath
import math as pymath
def call_csin(x):
    return cmath.sin(x)
def call_pysin(x):
    return pymath.sin(x)

#### Definition files和C(C++)头文件的相似处：
- 他们都声明了C-level constructs，以被外部代码使用
- 他们都允许我们将一个大文件分解成几个组件
- 他们都为一个实现（implementation）声明了public C-level interface

### IV. Include Files and the include Statement
Include files和<font face='consolas'>include</font>语句提供了一种实现平台无关设计目标的方法。他们实现了源码级别的inclusion，这与<font face='consolas'>cimport</font>不同，<font face='consolas'>cimport</font>可以看做编译时的<font face='consolas'>import</font>语句，操作的是命名空间。

In [ ]:
IF UNAME_SYSNAME == "Linux":
    include "linux.pxi"
ELIF UNAME_SYSNAME == "Darwin":
    include "darwin.pxi"
ELIF UNAME_SYSNAME == "Windows":
    include "windows.pxi"

### V. Organizing and Compiling Cython Modules Inside Python Packages
<font color='ff22aa' face='微软雅黑'>page 130</font>
- 确定Python Project中要使用Cython进行优化的部分。
- 将要优化的*.py*文件改为*.pyx*文件，使用Cython改写，并抽取他们的public Cython declarations到对应的*.pxd*文件中。
    - 若一个definitoin file需要使用其他definitoin file中的declaration，<font face='consolas'>cimport</font>相应definiton file中的相应declaration。
- 编译Cython源文件，生成扩展模块